From here we can found a fast way to start a dedicated server from google colaboratory (free).  
The maximum runtime of the server at here is about 12 hours (or less).  

If you want to save states of the game, please download your cluster' folders normally on the left panel (Files).

You will read and run cell by cell to get more information and do some additional stuff (such as replace your server token, ...).

In the furture, you totally customize all things, that will be change server cluster, change seeds, auto save all file to google drive to save time (mount GD and change $PR_DIR to exactly location u want), add mods,...

# Original tutorial post (by klei 4rum)

https://forums.kleientertainment.com/forums/topic/64441-dedicated-server-quick-setup-guide-linux/

Make sure this machine support x86 environment (32bit)

In [ ]:
!sudo dpkg --add-architecture i386
!sudo apt-get update

!sudo apt-get install libstdc++6:i386 libgcc1:i386 libcurl4-gnutls-dev:i386

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

## Connect to working dir (can be located on google drive)

In [ ]:
PR_DIR="/content/"
%cd $PR_DIR

/content


## install steamcmd

In [ ]:
!mkdir -p steamcmd
%cd steamcmd

/content/steamcmd


In [ ]:
!wget "https://steamcdn-a.akamaihd.net/client/installer/steamcmd_linux.tar.gz"

!tar -xvzf steamcmd_linux.tar.gz
!rm steamcmd_linux.tar.gz

!chmod -R 0755 *

%cd $PR_DIR

--2021-07-04 07:27:03--  https://steamcdn-a.akamaihd.net/client/installer/steamcmd_linux.tar.gz
Resolving steamcdn-a.akamaihd.net (steamcdn-a.akamaihd.net)... 23.40.205.34, 23.40.205.75
Connecting to steamcdn-a.akamaihd.net (steamcdn-a.akamaihd.net)|23.40.205.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2428561 (2.3M) [application/octet-stream]
Saving to: ‘steamcmd_linux.tar.gz’

steamcmd_linux.tar. 100%[===================>]   2.32M  --.-KB/s    in 0.06s   

2021-07-04 07:27:03 (35.6 MB/s) - ‘steamcmd_linux.tar.gz’ saved [2428561/2428561]

steamcmd.sh
linux32/steamcmd
linux32/steamerrorreporter
linux32/libstdc++.so.6
linux32/crashhandler.so
/content


## Mods

My mods file for my server is `https://drive.google.com/uc?id=1wEqvUR9W24amYqJbCnfm4KfZGEgKo9fG`, hence you can replace by your own.

In [ ]:
%cd $PR_DIR
!mkdir -p dontstarvetogether_dedicated_server
%cd dontstarvetogether_dedicated_server/

!gdown --id 1wEqvUR9W24amYqJbCnfm4KfZGEgKo9fG

/content
/content/dontstarvetogether_dedicated_server
Downloading...
From: https://drive.google.com/uc?id=1wEqvUR9W24amYqJbCnfm4KfZGEgKo9fG
To: /content/dontstarvetogether_dedicated_server/mods.zip
301MB [00:03, 83.0MB/s]


Extract mods file to mods folder

In [ ]:
!unzip -q -o mods.zip
!ls

mods  mods.zip


## Klei and server

In [ ]:
%cd $PR_DIR 
!mkdir -p DoNotStarveTogether

/content


In [ ]:
%cd $PR_DIR/DoNotStarveTogether

/content/DoNotStarveTogether


Download my own server. You can use this too becasue there is only a world seed (Start at Day1). This is a `shipwrecked extended` world create by [Tropical Experience | The Volcano Biome](https://steamcommunity.com/sharedfiles/filedetails/?id=1505270912) mod.

In [ ]:
%cd $PR_DIR/DoNotStarveTogether

#!wget -qO- https://drive.google.com/uc?id=1egHeOeZWrUc4WFv_iqMc5m8HZ0YEfvpF > MyDediServer2.zip
!wget -q https://cdn.discordapp.com/attachments/817270986423664670/859248019118358588/MyDediServer.zip

!unzip -q -o MyDediServer.zip
!rm MyDediServer.zip

!ls

/content/DoNotStarveTogether
modoverrides.lua  MyDediServer	MyDediServer2.zip


Please note that the name of dedicated server is `MyDediServer`, you need to rename again to make sure this match together.

After that, please use your own `SECRET TOKEN` (can be gen from [Klei Server page](https://accounts.klei.com/account/game/servers?game=DontStarveTogether)) instead of my token, then replace your token here:

In [ ]:
YOUR_TOKEN_PLS = "PLEASE REPLACE ME INSIDE THE QUOTE"

assert YOUR_TOKEN_PLS != "PLEASE REPLACE ME INSIDE THE QUOTE"

Make sure the previnous cell run complete, then followup:

In [ ]:
with open('MyDediServer/cluster_token.txt', 'wt') as f:
    f.write(YOUR_TOKEN_PLS)

ur_saved_token = !cat MyDediServer/cluster_token.txt
print(ur_saved_token[0])

assert ur_saved_token[0] != "PLEASE REPLACE ME INSIDE THE QUOTE"

Copy server files to `.klei` folders.

In [ ]:
%cd ~

!mkdir -p .klei
!mkdir -p .klei/DoNotStarveTogether/

!cp -r $PR_DIR/DoNotStarveTogether/* ~/.klei/DoNotStarveTogether

!ls $PR_DIR/DoNotStarveTogether/

/root
modoverrides.lua  MyDediServer	MyDediServer2.zip


## RUN

download auto script

In [ ]:
%cd $PR_DIR

!git clone https://github.com/viplazylmht/DstDedicated.git

!cp DstDedicated/run_dedicated_servers.sh .

/content
Cloning into 'DstDedicated'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 29 (delta 11), reused 22 (delta 7), pack-reused 0
Unpacking objects: 100% (29/29), done.


Then just run this script, almost done.  

Please note that we need to download DST game (abount 2.3GB) when the first time run, and when you factory reset runtime. 

In [ ]:
%cd $PR_DIR
!chmod u+x run_dedicated_servers.sh
!./run_dedicated_servers.sh $PR_DIR MyDediServer

Streaming output truncated to the last 5000 lines.
Master: [00:01:14]: Mod: workshop-1505270912 ( Tropical Experience Return of Them)	    snakeskinhat	
Master: [00:01:14]: Mod: workshop-1505270912 ( Tropical Experience Return of Them)	    oxhat	
Master: [00:01:14]: Mod: workshop-1505270912 ( Tropical Experience Return of Them)	    aerodynamichat	
Master: [00:01:14]: Mod: workshop-1505270912 ( Tropical Experience Return of Them)	    brainjellyhat	
Master: [00:01:14]: Mod: workshop-1505270912 ( Tropical Experience Return of Them)	    disguisehat	
Master: [00:01:14]: Mod: workshop-1505270912 ( Tropical Experience Return of Them)	    gashat	
Master: [00:01:14]: Mod: workshop-1505270912 ( Tropical Experience Return of Them)	  Registering prefab file: prefabs/parrot_pirate	
Master: [00:01:14]: Mod: workshop-1505270912 ( Tropical Experience Return of Them)	    parrot_pirate	
Master: [00:01:14]: Mod: workshop-1505270912 ( Tropical Experience Return of Them)	  Registering prefab file: prefabs/d

In [ ]:
# LOG
# Master: [00:00:26]: World generated on build 469426 with save version: 5.065, using seed: 1624866836
# Caves:  [00:00:32]: World generated on build 469426 with save version: 5.065, using seed: 1624866835	

If you want to stop this dedicated server, just go to the toolbar, find `Runtime` option, and click `Interrupt execute`.

## Backup save game

In [ ]:
%cd ~/.klei/DoNotStarveTogether

/root/.klei/DoNotStarveTogether


In [ ]:
!zip -r ServerBackup.zip MyDediServer

  adding: MyDediServer/ (stored 0%)
  adding: MyDediServer/blocklist.txt (deflated 1%)
  adding: MyDediServer/cluster.ini (deflated 33%)
  adding: MyDediServer/Caves/ (stored 0%)
  adding: MyDediServer/Caves/leveldataoverride.lua (deflated 71%)
  adding: MyDediServer/Caves/server_chat_log.txt (deflated 78%)
  adding: MyDediServer/Caves/worldgenoverride.lua (deflated 26%)
  adding: MyDediServer/Caves/save/ (stored 0%)
  adding: MyDediServer/Caves/save/client_temp/ (stored 0%)
  adding: MyDediServer/Caves/save/world_presets/ (stored 0%)
  adding: MyDediServer/Caves/save/profile (deflated 15%)
  adding: MyDediServer/Caves/save/modindex (deflated 20%)
  adding: MyDediServer/Caves/save/shardindex (deflated 68%)
  adding: MyDediServer/Caves/save/shardindex_time (deflated 12%)
  adding: MyDediServer/Caves/save/session/ (stored 0%)
  adding: MyDediServer/Caves/save/session/AA6E9884A4CFA3E5/ (stored 0%)
  adding: MyDediServer/Caves/save/session/AA6E9884A4CFA3E5/0000000027 (deflated 85%)
  addin

In [ ]:
!mv ServerBackup.zip $PR_DIR

Goo, download your ServerBackup.zip, Done.